In [ ]:
#! jupyter nbconvert --to script L-R.ipynb

# Importing Libraries

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.run import Run
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib

# Authentication

In [ ]:
auth = InteractiveLoginAuthentication(tenant_id="864324a0-fbca-44b4-a025-ee7be7426f0b")
ws = Workspace.from_config(auth=auth)

# Delete Experiment

In [ ]:
# experiment_id = Experiment(ws, "azure_version").id

# Experiment.delete(ws, experiment_id)

# Environment setup

In [ ]:
conda_dep = CondaDependencies().create(conda_packages=["scikit-learn==1.4.2"])
myenv = Environment(name="myenv2")
myenv.python.conda_dependencies = conda_dep

# Data

In [ ]:
celsius_q = np.array([-40, -10, 0, 8, 15, 22, 38], dtype=float)
fahrenheit_a = np.array([-40, 14, 32, 46, 59, 72, 100], dtype=float)

# Model training

In [ ]:
X = celsius_q.reshape(-1, 1)
model = LinearRegression()
model.fit(X, fahrenheit_a)
joblib.dump(model, "model.pkl")

# Azure ML experiment setup

In [ ]:
experiment_name = "Linear-Regression-7"
experiment = Experiment(workspace=ws, name=experiment_name)

# Script run configuration

In [ ]:
script_config = ScriptRunConfig(
    source_directory=".",
    script="L-R.py",  # Update script name if necessary
    compute_target="OHMS-ML",  # Update compute target if necessary
    environment=myenv,
)

# Experiment submission

In [37]:
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)